In [1]:
from typing import List
import glob
import os
import re
import shutil
from tqdm import tqdm
from src.models.video import TimeSformerInferencing
from src.post_processing import process_football_events, cfg
from src.preprocessing.video import normalize_path
from src.post_processing.utils import read_json, joint_event, concat_videos
from typing import List

inferencing =TimeSformerInferencing(
    num_frames= 30,
    num_classes = 17,
    game_name = './tv360/video_classification/gir_vs_feynord',
    image_extractor="fcakyon/timesformer-large-finetuned-k400",
    checkpoint = './ckpt/ds_tv360_model_timesformer_nepochs_2_nframes_30.pth')

results = inferencing.run(method='from_middle', spacing = 30)
game, event2timestamp = process_football_events(json_path = results, rank = cfg.rank)
#2m388

2025-03-26 06:43:35 UTC+7 | WARNING  | src.logger.log_handler:warning:109 | This path is not exist, use default constants
d:\projects\v2v\v5\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of TimesformerModel were not initialized from the model checkpoint at facebook/timesformer-base-finetuned-k600 and are newly initialized because the shapes did not match:
- timesformer.embeddings.time_embeddings: found shape torch.Size([1, 8, 768]) in the checkpoint and torch.Size([1, 30, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 99/99 [02:31<00:00,  1.53s/it]


In [2]:
game, event2timestamp = process_football_events(json_path = results, rank = cfg.rank)

In [4]:
base_dir = "D:\\projects\\v2v\\v5\\tv360\\video_classification"
def merge_events(base_dir, game):
    # basedir = "F:/video_classification"
    game_path = f"{base_dir}/{game}"
    highlights = []
    for event, timestamps in tqdm(event2timestamp.items()):
        for timestamp in timestamps:
            print(timestamp)
            if isinstance(timestamp, tuple):
                # /visual/*mp4
                video1 = normalize_path(glob.glob(f"{game_path}/chunk_{timestamp[0]}/*.mp4")[0])
                video2 =  normalize_path(glob.glob(f"{game_path}/chunk_{timestamp[1]}/*.mp4")[0])
                concat_videos([video1, video2], f"{event}_{timestamp[0]}_{timestamp[1]}.mp4")
                highlights.append(f"{event}_{timestamp[0]}_{timestamp[1]}.mp4")
            elif isinstance(timestamp, int):
                video = normalize_path(glob.glob(f"{game_path}/chunk_{timestamp}/*.mp4")[0])
                concat_videos([video], f"{event}_{timestamp}.mp4")
                highlights.append(f"{event}_{timestamp}.mp4")
            else:
                pass

    highlights = sorted(highlights, key=lambda x: list(map(int, re.findall(r'\d+', x)))[0])
    target_folder = "sample"
    os.makedirs(target_folder, exist_ok=True)
    for file in highlights:
        if os.path.exists(file):  # Ensure the file exists before moving
            shutil.move(file, os.path.join(target_folder, file))
    
    
    return highlights

highlights=merge_events(base_dir = base_dir, game = game)

  0%|          | 0/6 [00:00<?, ?it/s]

(22, 23)
Đã nối video thành công: Goal_22_23.mp4
(35, 36)
Đã nối video thành công: Goal_35_36.mp4
(50, 51)
Đã nối video thành công: Goal_50_51.mp4
(68, 69)
Đã nối video thành công: Goal_68_69.mp4
(75, 76)
Đã nối video thành công: Goal_75_76.mp4
(80, 81)
Đã nối video thành công: Goal_80_81.mp4
(88, 89)


 17%|█▋        | 1/6 [00:01<00:07,  1.55s/it]

Đã nối video thành công: Goal_88_89.mp4
(97, 98)
Đã nối video thành công: Goal_97_98.mp4


 50%|█████     | 3/6 [00:01<00:01,  2.22it/s]

...

Đã nối video thành công: Foul_5.mp4
25
Đã nối video thành công: Foul_25.mp4
(24, 26)
Đã nối video thành công: Substitution_24_26.mp4
(77, 78)


 83%|████████▎ | 5/6 [00:02<00:00,  2.18it/s]

Đã nối video thành công: Substitution_77_78.mp4
9
Đã nối video thành công: Shots_9.mp4
(17, 18)
Đã nối video thành công: Shots_17_18.mp4
46
Đã nối video thành công: Shots_46.mp4
67


100%|██████████| 6/6 [00:03<00:00,  1.73it/s]

Đã nối video thành công: Shots_67.mp4
79
Đã nối video thành công: Shots_79.mp4


In [3]:
import numpy as np
import skvideo.measure
import cv2
import skvideo.io
import os
from tqdm import tqdm
from src.post_processing.utils import filter_highlight_scenes, cut_video_by_frame, merge_videos
np.float = float    
np.int = int   #module 'numpy' has no attribute 'int'
np.object = object    #module 'numpy' has no attribute 'object'
np.bool = bool 

filelist = [f"sample/{i}" for i in os.listdir("sample")]
for filename in tqdm(filelist):
    videodata = skvideo.io.vread(filename)
    scene_lum_idx = skvideo.measure.scenedet(videodata, method='histogram', parameter1=1.0)
    frame_cut = filter_highlight_scenes(scene_lum_idx)
    cut_video_by_frame(video_path = filename, frame_position = frame_cut, middle_threshold= 0.22)

  0%|          | 0/18 [00:00<?, ?it/s]

626
735


  6%|▌         | 1/18 [02:18<39:11, 138.32s/it]

Frame 680 nằm ở giữa video (frame 504-1296), giữ nguyên video. Kết quả: sample\Cards_62_full.mp4


  6%|▌         | 1/18 [02:48<47:40, 168.27s/it]


KeyboardInterrupt: 

In [ ]:
# Ghép nối sử dụng phương pháp concat (nhanh, giữ nguyên chất lượng)
merged_video = merge_videos(highlights, "output_merged.mp4", method="concat")

# Nếu có lỗi với phương pháp concat (ví dụ: codec khác nhau), thử phương pháp filter_complex
if not merged_video:
    merge_videos(highlights, "output_merged.mp4", method="filter_complex")